In [12]:
from G_model import GProblem
from mesa.batchrunner import BatchRunnerMP
import pandas as pd
import numpy as np
from functools import partial

pd.set_option("precision", 3)


# Grim et al - replication runs

Grim et al run 20 different models, each 100 times. Here, 500 iterations are run, which take about 1 minute per iteration on a 4-core 2.4 Ghz laptop.

In [18]:
fixed_params = {"n": 2000,
               "k": 3,
               "N_agents": 10,
               "l": 12
               }
variable_params = {
                   "smoothness": list(range(21))
                   }

np.random.seed(1)

batch_run = BatchRunnerMP(GProblem,
                        4,
                        variable_parameters = variable_params,
                        fixed_parameters = fixed_params,
                        iterations=500,
                        max_steps=100,
                        model_reporters={"agent_descriptives": lambda m: m.agent_descriptives,
                        "best_solution": lambda m: m.best_solution})
batch_run.run_all()

42it [02:58,  4.25s/it]


({(0,
   2000,
   3,
   10,
   12,
   0): OrderedDict([('agent_descriptives',
                {'random': {'worst_agent': 83.9069053731275,
                  'average_agent': 85.27257962884897,
                  'top_agent': 86.74832154186936,
                  'team_average': 85.1255663885307,
                  'NPdiversity': 0.8888888888888888,
                  'Cdiversity': 0.8333333333333334},
                 'best': {'worst_agent': 83.9069053731275,
                  'average_agent': 85.27257962884897,
                  'top_agent': 86.74832154186936,
                  'team_average': 86.49903160387889,
                  'NPdiversity': 0.8296296296296296,
                  'Cdiversity': 0.5833333333333334}}),
               ('best_solution',
                {'random': 94.85613548726522, 'best': 79.6489328475812})]),
  (0,
   2000,
   3,
   10,
   12,
   1): OrderedDict([('agent_descriptives',
                {'random': {'worst_agent': 84.37950447369293,
                  'average

In [19]:
res = batch_run.get_model_vars_dataframe()

# Turn dictionaries into columns
res = pd.concat([res.drop(["best_solution"], axis = 1), res.best_solution.apply(pd.Series).add_suffix("_solution")], axis = 1)

def renamer(col, prefix):
    if col.endswith("agent"):
        return col
    else:
        return prefix + col

res_random = res.agent_descriptives.apply(pd.Series).random.apply(pd.Series).rename(mapper = partial(renamer, prefix = "random_"), axis = "columns")

res_best = res.agent_descriptives.apply(pd.Series).best.apply(pd.Series).rename(mapper = partial(renamer, prefix = "best_"), axis = "columns")

res = pd.concat([res.drop(["agent_descriptives"], axis=1), res_best, res_random[res_random.columns[pd.Series(res_random.columns).str.startswith('random_')]]], axis=1)

res["run_id"] = res.reset_index().index
res = res.rename(columns={"best_agent": "top_agent"})

#Pivot so that random and best groups can be easily compared
col_names = res.columns.values.tolist()

def check_var(col_name):
    return not(col_name.find("random_") != -1 or col_name.find("best_") != -1)


id_cols = list(filter(check_var, col_names))

out = pd.melt(res, id_cols)

out = out.join(out.variable.str.split("_", expand = True)).rename(columns={0:"team_type"}).pivot_table(index=id_cols + ["team_type"], columns=[1], values="value").reset_index()


In [20]:
res.head()

,smoothness,Run,n,k,N_agents,l,random_solution,best_solution,worst_agent,average_agent,top_agent,best_team_average,best_NPdiversity,best_Cdiversity,random_team_average,random_NPdiversity,random_Cdiversity,run_id
0,0,2000,2000,3,10,12,94.856,79.649,83.907,85.273,86.748,86.499,0.830,0.583,85.126,0.889,0.833,0
23,10,2000,2000,3,10,12,87.883,61.948,67.193,70.375,72.733,72.660,0.615,0.750,70.918,0.904,1.000,1
24,12,2000,2000,3,10,12,82.096,98.377,66.149,68.553,70.514,70.449,0.674,0.500,68.252,0.926,0.917,2
25,12,2000,2000,3,10,12,93.401,80.308,67.706,71.790,74.249,74.170,0.741,0.500,71.805,0.904,1.000,3
26,13,2000,2000,3,10,12,11.273,78.801,57.450,60.533,62.602,62.519,0.607,0.583,60.902,0.963,0.917,4


In [5]:
out.to_csv("model_results.csv")

In [8]:
#Performance and diversity of best versus random teams
out[["team_type", "N_agents", "l", "solution", "NPdiversity", "smoothness"]].groupby(["team_type", "N_agents", "l", "smoothness"]).describe().loc[:, (slice(None), ["mean", "std"])]


1                                solution         NPdiversity       
                                     mean     std        mean    std
team_type N_agents l  smoothness                                    
best      10       12 0            95.856   5.377       0.812  0.069
                      1            89.615  10.332       0.783  0.004
                      2            85.849  12.438       0.765  0.028
                      3            88.729  10.755       0.765  0.017
                      4            73.051  27.993       0.706  0.041
                      5            60.128   8.501       0.714  0.038
                      6            83.902  26.017       0.689  0.020
                      7            60.441  12.593       0.627  0.035
                      8            85.273  13.517       0.607  0.084
                      9            65.528  12.517       0.659  0.063
                      10           53.794  24.124       0.664  0.081
                      11           46.916  31.123       0.709  0.023
                      12           59.078   5.925       0.699  0.045
                      13           55.023  10.575       0.630  0.038
                      14           72.759  14.744       0.721  0.043
                      15           59.060  31.867       0.598  0.009
                      16           57.195  23.896       0.691  0.037
                      17           69.515  14.429       0.684  0.024
                      18           66.846  40.008       0.669  0.004
                      19           81.659  10.455       0.674  0.034
                      20           62.695  23.187       0.674  0.063
random    10       12 0            99.152   0.883       0.921  0.023
                      1            87.290   8.609       0.909  0.011
                      2            97.198   2.572       0.923  0.028
                      3            88.380  10.266       0.906  0.024
                      4            90.682   6.051       0.914  0.004
                      5            76.693  23.814       0.906  0.011
                      6            86.884  19.317       0.919  0.020
                      7            80.076   3.218       0.919  0.013
                      8            81.421  16.448       0.906  0.011
                      9            76.472  18.923       0.923  0.011
                      10           73.387   6.617       0.911  0.015
                      11           66.946   6.790       0.923  0.011
                      12           73.978  22.151       0.916  0.019
                      13           72.568   9.056       0.946  0.019
                      14           74.001   5.296       0.919  0.020
                      15           69.232  15.745       0.919  0.007
                      16           94.941   4.191       0.936  0.015
                      17           73.176  13.028       0.923  0.043
                      18           86.785  10.847       0.926  0.030
                      19           78.798  24.128       0.926  0.020
                      20           68.062  22.791       0.906  0.043

In [9]:
#Describe agents
res[["worst_agent", "top_agent", "best_team_average", "random_team_average", "N_agents", "l", "smoothness"]].groupby(["N_agents", "l", "smoothness"]).describe().loc[:, (slice(None), ["mean", "std"])]

worst_agent        top_agent        best_team_average  \
                              mean    std      mean    std              mean   
N_agents l  smoothness                                                         
10       12 0               83.633  0.650    86.480  0.766            86.284   
            1               77.580  1.366    83.981  0.412            83.764   
            2               72.085  0.854    82.781  1.039            82.588   
            3               69.139  0.952    81.348  0.699            81.026   
            4               68.966  4.702    79.242  3.088            79.000   
            5               65.943  1.659    76.207  0.603            75.958   
            6               67.024  2.062    75.099  2.460            74.986   
            7               66.247  1.848    73.631  3.077            73.522   
            8               64.722  1.233    72.481  1.020            72.356   
            9               65.358  2.348    71.858  2.664            71.724   
            10              63.980  3.385    69.541  3.031            69.457   
            11              63.272  1.364    68.723  0.872            68.664   
            12              67.143  4.233    71.572  2.742            71.471   
            13              67.907  0.967    72.340  0.616            72.262   
            14              65.271  1.882    69.963  1.134            69.903   
            15              63.701  4.708    67.026  5.118            66.987   
            16              64.654  2.418    68.202  2.595            68.145   
            17              64.182  1.051    68.138  1.117            68.106   
            18              66.317  3.196    70.188  3.265            70.150   
            19              66.255  3.477    69.241  2.753            69.195   
            20              64.394  1.945    67.208  2.490            67.170   

                              random_team_average         
                          std                mean    std  
N_agents l  smoothness                                    
10       12 0           0.652              85.102  0.640  
            1           0.439              81.366  0.666  
            2           1.134              78.617  1.723  
            3           0.636              76.189  1.037  
            4           3.152              75.352  4.461  
            5           0.685              71.769  0.830  
            6           2.478              70.113  1.838  
            7           3.086              70.029  2.874  
            8           0.996              68.889  0.505  
            9           2.609              68.959  2.516  
            10          3.002              67.171  3.162  
            11          0.876              66.434  0.879  
            12          2.749              69.844  2.817  
            13          0.582              69.975  1.134  
            14          1.136              68.067  1.194  
            15          5.122              65.749  5.054  
            16          2.587              66.652  2.544  
            17          1.123              66.436  1.209  
            18          3.254              68.909  3.512  
            19          2.765              68.026  2.988  
            20          2.474              65.992  2.285